1. Write a approx 750-800 (3PAGE) word description of: 
    - Chatbot application (e.g., use-case, domain of operation).
    - Atleast 3 different test cases that clearly illustrate chatbot behavior.
    - How you connected your work to lecture material 
    - Advanced techniques that you used

If you are submitting a PDF, please mention that here. 

-----------------------------------------------------
##### PASTE YOUR TEXT HERE
Submitted as PDF - Megamind Report
-----------------------------------------------------

------------
2. CODE
-----------

In [12]:
# PASTE THE CODE 
# Make this easy for user testing - Running this cell should load all code for Chatbot.

import re
import random
import pandas as pd
import json

# read the JSON file
def read_json(fileName):
    read = pd.read_json(fileName)
    return(read)

# function that returns rgx2int
def tags2intent(json_data):
    rgx2int = {}
    for intent in json_data["intents"]:
        for pattern in intent["patterns"]:
            rgx2int[r"{}".format(re.escape(pattern))] = (intent["tag"], intent.get("match_type", "partial"))
    return rgx2int


# function that returns int2res
def intent2response(json_data):
    int2res = {}
    for intent in json_data["intents"]:
        int2res[intent["tag"]] = intent["responses"]
    return int2res

# function that detects intent and its match type
def detect_intent(user_input, rgx2int):
    for pattern, (intent, match_type) in rgx2int.items():
        if match_type == "full" and re.fullmatch(pattern, user_input, re.IGNORECASE):
            return intent
        elif match_type == "partial" and re.search(pattern, user_input, re.IGNORECASE):
            return intent
    return "default"

# function that does chatbot_response
def chatbot_response(detected_intent, int2res, memory):
    if detected_intent in int2res:
        responses = int2res[detected_intent]
        
        # responses based on memory
        if detected_intent == "colour" and memory["colour"] == "nothing yet":
            memory["awaiting_colour"] = True
            response = random.choice(responses)
        elif detected_intent == "colour" and memory["colour"]:
            response = random.choice([
                f"Green is my favourite, but I know you like {memory['colour']}!",
                f"I like green! You mentioned before that you like {memory['colour']}."
            ])
        elif detected_intent == "food" and memory["food"] == "nothing yet":
            memory["awaiting_food"] = True
            response = random.choice(responses)
        elif detected_intent == "food" and memory["food"]:
            response = random.choice([
            f"Speghetti is my favourite, but I know you like {memory['food']}!",
            f"I love speghetti! You mentioned before that you like {memory['food']}."
            ])
        else:
            response = random.choice(responses)
        
        # string substitution for name, colour and food
        if "{name}" in response and memory["name"]:
            response = response.format(name=memory["name"])

        if "{food}" in response and memory["food"]:
            response = response.format(food=memory["food"])

        if "{colour}" in response and memory["colour"]:
            response = response.format(colour=memory["colour"])

        return response
    else:
        return "What else would you like to know?"


#update memory dynamically
def update_memory(user_input, memory):
    #detecting and saving name without prompt
    name_match = re.search(r"\b(?:my name is|call me) (\w+)", user_input, re.IGNORECASE)
    if name_match:
        memory["name"] = name_match.group(1)
        print(f"Megamind: Nice to meet you {memory['name']}!")
        return

    # detecting and saving colour without prompt
    colour_match = re.search(r"\b(?:i like the colour|my favourite colour is|my fave colour is) (\w+)", user_input, re.IGNORECASE)
    if colour_match:
        memory["colour"] = colour_match.group(1)
        print(f"Megamind: Oh, {memory['colour']} is a beautiful color!")
        return
            
    # detecting and saving colour with prompt from bot
    if memory.get("awaiting_colour"):
        # if user uses a direct answer
        direct_colour_match = re.match(r"^\b(\w+)[.!?]?$", user_input, re.IGNORECASE)
        if direct_colour_match:
            memory["colour"] = direct_colour_match.group(1)
            memory["awaiting_colour"] = False
            print(f"Megamind: Oh, {memory['colour']} is a beautiful color!")
            return

        # if user uses a full sentence
        colour_match = re.search(r"\b(?:i like|i like the colour|my favourite colour is|my fave colour is) (\w+)", user_input, re.IGNORECASE)
        if colour_match:
            memory["colour"] = colour_match.group(1)
            memory["awaiting_colour"] = False
            print(f"Megamind: Oh, {memory['colour']} is a beautiful color!")
            return

    # detecting and saving food without prompt
    food_match = re.search(r"\b(?:my favourite food is|my fave food is|i love) (\w+)", user_input, re.IGNORECASE)
    if food_match:
        memory["food"] = food_match.group(1)
        print(f"Megamind: Yum! I’ll remember that you like {memory['food']}.")
        return
            
    # detecting and saving food with prompt from bot
    if memory.get("awaiting_food"):
        # if user uses a direct answer
        direct_food_match = re.match(r"^\b(\w+)[.!?]?$", user_input, re.IGNORECASE)
        if direct_food_match:
            memory["food"] = direct_food_match.group(1)
            memory["awaiting_food"] = False
            print(f"Megamind: Yum! I’ll remember that you like {memory['food']}.")
            return

        # if user uses a full sentence
        food_match = re.search(r"\b(?:i like|my favourite food is|my favourite food are|my fave food is|i love) (\w+)", user_input, re.IGNORECASE)
        if food_match:
            memory["food"] = food_match.group(1)
            memory["awaiting_food"] = False
            print(f"Megamind: Yum! I’ll remember that you like {memory['food']}.")
            return

# function to remove special characters, keeping only alphanumeric characters and spaces from user input
def preprocess_input(user_input):
    cleaned_input = re.sub(r"[^a-zA-Z0-9\s]", "", user_input)
    return cleaned_input.lower().strip()


# main chatbot loop
def chat():
    # load JSON data
    intents_data = read_json("json/intents.json")

    # create the regex to intent and intent to response dictionaries
    rgx2int = tags2intent(intents_data)
    int2res = intent2response(intents_data)

    # initialize memory
    memory = {"name": "there", "colour": "nothing yet", "food": "nothing yet",  "awaiting_name": False,  "awaiting_colour": False, "awaiting_food": False}
    
    print("Megamind: Hello! Ask me anything! Type 'exit' to end the chat.")
    while True:
        user_input = input("You: ")

        # Exit condition
        if re.search(r"\b(exit|quit|bye|goodbye)\b", user_input, re.IGNORECASE):
            print("Megamind: Goodbye! Have a great day!")
            break

        # detect intent
        detected_intent = detect_intent(preprocess_input(user_input), rgx2int)

        # update memory based on user input
        update_memory(user_input, memory)

        # chatbot response
        print(f"Megamind: {chatbot_response(detected_intent, int2res, memory)}")

# Run the chatbot
chat()

Megamind: Hello! Ask me anything! Type 'exit' to end the chat.


You:  hi


Megamind: Nice to meet you there! How are you?


You:  bye


Megamind: Goodbye! Have a great day!


In [13]:
# ATLEAST 3 INTERESTING RESULTS - AND FUNCTION CALLS TO GENERATE THEM

#Interesting result 1
# Megamind: Hello! Ask me anything! Type 'exit' to end the chat.
# You:  Hello!
# Megamind: Hi there, how can I help?
# You:  How can you help?
# Megamind: I can tell you an interesting fact! Try asking me for one.
# You:  Tell me a super interesting fact!
# Megamind: There are no seagulls in Hawaii.
# You:  Thats pretty interesting!
# Megamind: Thank you!

#Function calls:
#detect_intent(user_input, rgx2int) - detects greeting, then help, then fact, then thanks intent
#chatbot_response(detected_intent, int2res, memory) - responds with appropriate random choice of responses based in intent


#Interesting result 2
# You:  What is my favourite food?  
# Megamind: Your favourite food is nothing yet!
# You:  What is your favourite food?
# Megamind: My favourite food is spaghetti! Whats yours?
# You:  Meatballs!
# Megamind: Yum! I’ll remember that you like Meatballs.
# Megamind: What else would you like to know?
# You:  What is your favourite colour?
# Megamind: My favourite colour is green! Whats yours?
# You:  I like the colour blue!
# Megamind: Oh, blue is a beautiful color!
# Megamind: What else would you like to know?
# You:  Do you remember what my favourite food was?
# Megamind: Of course i remember your favourite food! It's Meatballs!
# You:  My favourite food is pizza!
# Megamind: Yum! I’ll remember that you like pizza.
# Megamind: What else would you like to know?
# You:  What is your favourite food?
# Megamind: I love speghetti! You mentioned before that you like pizza.

#Function calls:
#detect_intent(user_input, rgx2int) - detects userfood, then botfood, then botcolour, then userfood again and finally botfood
#update_memory(user_input, memory) - updates memory with memory["food"] = "Meatballs", then with memory["colour"] = "blue", then replaces memory["food"] with = "pizza"
#chatbot_response(detected_intent, int2res, memory) - responds with appropriate random choice of responses based on intent and memory


#Interesting result 3
# You:  My name is Jeremi
# Megamind: Nice to meet you Jeremi!
# Megamind: What else would you like to know?
# You:  What's your name?
# Megamind: You can call me Megamind!
# You:  Nice to meet you Megamind!
# Megamind: Nice to meet you Jeremi! How are you?
# You:  I'm doing well!
# Megamind: Thats great to hear!
# You:  How are you doing?
# Megamind: I'm great! Thanks for asking.

#Function calls:
#detect_intent(user_input, rgx2int) - detects username, then botname then greeting, then goodmood(for user), then botmood
#update_memory(user_input, memory) - updates memory with memory["name"] = "Jeremi" when username was detected
#chatbot_response(detected_intent, int2res, memory) - responds with appropriate random choice of responses based on intent and memory


3. Reflect on project management (approx 250 words) 
    - Timeliness: Reflect on how consistently you made an effort meet deadlines
    - Organization: Reflect on how managed the code-base as the project size increased

-----------------------------------------------------
Chatbot project management reflection:

Most of the progress on the chatbot was made in and after labs associated with its development, where I would consistently try to implement the features discussed into my own chatbot. I would prioritize completing lab work first before any implementation to make sure I understood exactly what sort of features I could add. I would then create a separate chatbot using only what was in lab. This led to many different iterations of chatbots, mainly one with a memory system, and another with a file read system.  As the project grew it became more difficult to meet these deadlines, with lab work from sessions occasionally spilling into each other, leaving me slightly behind with the newest implementation. This however was quickly overcome by simply adjusting my schedule to leave more time for chatbot development.

Thanks to jupyter notebooks easy code block structure, it was quite easy to organise my code, where I would usually reserve a block for a single function. This made the readability and navigation of the code much better, also improving debugging thanks to being able to run separate blocks of code independent of each other. This meant that I had pretty much no issues as the project size increased, only occasionally when closely related functions were in separate blocks, which mainly involved the main chatbot loop since that’s where all the functions are being called and used in tandem with each other. I addressed this by making sure to consistently add comments during development along with clear variable names.


-----------------------------------------------------


4. Process reflection (approx 200 words) - Discuss the week by week iterative development of your chatbot.
    - Describe each week of chatbot devleopment.
    - What was the feedback you received? How did you work on the feedback to improve chatbot? What new features did you add? 

-----------------------------------------------------

Process reflection:

During the first week I developed the main chatbot loop function, getting the simplest implementation of the chatbot ready – currently only capable of greeting you and terminating the program.

The second week ramped up a lot, with the implementation of complex dictionaries for user patterns and bot responses, along with a simple memory system, allowing the chatbot to remember the users name and colour. During the lab I struggled to find an elegant way to make the chatbot ask  the user unique questions, as it would always default to a single response – upon asking for help I was suggested the use of a random selection of responses, which I went to implement immediately. 

The third week I had created a new chatbot capable of reading json files, which would store our patterns and responses under unique tags as intents. To do this I used regex to match user input to intents and generate personalized chatbot responses based on those intents. 

During the last fourth week I looked into combining the knowledge I had gained from the previous weeks to make a chatbot with a large file of intents for enhanced interactivity, and a memory capable or remembering specific details about the user packaged in one. I had also developed a way for the bot to match if a pattern either exact or not.


-----------------------------------------------------
